In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
    .builder\
    .master('spark://spark-master:7077')\
    .appName('twitter_transformation')\
    .getOrCreate()
spark

In [4]:
df = spark.read.json('s3a://datalake/bronze/twitter_datascience')
df.toPandas()

,data,includes,meta,extract_date
0,"[(17, 55, 2023-11-07T04:32:10.781548+0000, [34...","([(2023-11-07T05:12:03.007567+0000, 45, User 1...","(1234567890abcdef,)",2023-11-07
1,"[(78, 35, 2023-11-10T21:24:44.937176+0000, [68...","([(2023-11-10T15:35:12.605861+0000, 26, User 1...","(1234567890abcdef,)",2023-11-10
2,"[(45, 36, 2023-11-06T00:41:17.611815+0000, [77...","([(2023-11-06T16:15:34.102194+0000, 83, User 1...","(1234567890abcdef,)",2023-11-06
3,"[(63, 99, 2023-11-09T20:25:48.937182+0000, [14...","([(2023-11-09T12:32:56.257276+0000, 99, User 1...",None,2023-11-09
4,"[(76, 62, 2023-11-08T05:33:31.208387+0000, [85...","([(2023-11-08T06:09:40.772983+0000, 47, User 1...",None,2023-11-08


In [15]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [6]:
from pyspark.sql import functions as f

In [ ]:
df.select(f.explode("data")).printSchema()

In [ ]:
df.select(f.explode("data")).show()

In [ ]:
df.select(f.explode("data").alias("tweets"))\
.select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

In [11]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

tweet_df.show()


+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       17|             55|2023-11-07T04:32:...| 52|        52|         38|         51|           33|Este é um tweet f...|
|       34|             82|2023-11-07T01:29:...| 71|        12|         18|         24|           61|Outro tweet fictí...|
|       53|              0|2023-11-07T22:55:...| 24|         0|         17|         43|           41|Um terceiro tweet...|
|       77|             29|2023-11-07T18:23:...| 98|        69|         24|         24|           48|Tweet fictício ge...|
|       35|             40|2023-11-07T23:05:...| 25|        67|         39|          5|           73|Tweet fictício cr...|
|       45|     

In [14]:
process_date = '2023-11-11'
tweet_df = tweet_df.withColumn('process_date', f.to_date(f.lit(process_date)))
tweet_df.show()

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|process_date|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+------------+
|       17|             55|2023-11-07T04:32:...| 52|        52|         38|         51|           33|Este é um tweet f...|  2023-11-11|
|       34|             82|2023-11-07T01:29:...| 71|        12|         18|         24|           61|Outro tweet fictí...|  2023-11-11|
|       53|              0|2023-11-07T22:55:...| 24|         0|         17|         43|           41|Um terceiro tweet...|  2023-11-11|
|       77|             29|2023-11-07T18:23:...| 98|        69|         24|         24|           48|Tweet fictício ge...|  2023-11-11|
|       35|             40|2023-11-07T23:05:...|

In [ ]:
tweet_df.show(5)

In [ ]:
df.select(f.explode('includes.users'))

In [ ]:
users = df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

In [ ]:
users.show(5)